In [8]:
!pip install webdriver-manager

Defaulting to user installation because normal site-packages is not writeable


In [194]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [240]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [241]:
job_title = "python-developer"
url = f"https://ph.linkedin.com/jobs/{job_title}-jobs"

In [242]:
driver.get(url)
driver.implicitly_wait(10)
skip_button = driver.find_element(By.XPATH, "//button[@aria-label='I-dismiss']")
skip_button.click()
job_list = driver.find_elements(By.XPATH, "//ul[@class='jobs-search__results-list']/li")

In [247]:
for job in job_list[:5]:
    card = job.find_element(By.XPATH, ".//a[@data-tracking-control-name='public_jobs_jserp-result_search-card']")
    url = card.get_attribute('href')
    job_title = job.find_element(By.XPATH, ".//h3[@class='base-search-card__title']").get_attribute('innerHTML').strip()
    try: 
        company = job.find_element(By.XPATH, ".//h4[@class='base-search-card__subtitle']/a").get_attribute('innerHTML').strip()
    except:
        company = job.find_element(By.XPATH, ".//h4[@class='base-search-card__subtitle']").get_attribute('innerHTML').strip()
    job.find_element(By.XPATH, ".//div").click()
    driver.implicitly_wait(10)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    print(soup.find_all("ul"))
    print('----')

[<ul class="switcher-tabs__list flex flex-1 items-stretch papabear:flex-col" role="tablist">
<li class="switcher-tabs__tab h-[44px] babymamabear:basis-1/2" role="presentation">
<button aria-controls="jobs-search-panel" aria-selected="true" class="switcher-tabs__button w-full h-full tab-md papabear:tab-vertical papabear:justify-start cursor-pointer tab-selected" data-switcher-type="JOBS" data-tracking-control-name="public_jobs_switcher-tabs-jobs-search-switcher" id="job-switcher-tab" role="tab">
                        Mga Trabaho
                    </button>
</li>
<li class="switcher-tabs__tab h-[44px] babymamabear:basis-1/2" role="presentation">
<button aria-controls="people-search-panel" aria-selected="false" class="switcher-tabs__button w-full h-full tab-md papabear:tab-vertical papabear:justify-start cursor-pointer" data-switcher-type="PEOPLE" data-tracking-control-name="public_jobs_switcher-tabs-people-search-switcher" id="people-switcher-tab" role="tab">
                        

In [210]:
print(BeautifulSoup(driver.page_source, 'html.parser').prettify())

<html lang="tl">
 <head>
  <meta content="d_jobs_guest_search" name="pageKey"/>
  <!-- -->
  <meta content="urlType=jserp_canonical_other;emptyResult=false" name="linkedin:pageTag"/>
  <meta content="tl_PH" name="locale"/>
  <meta data-app-version="2.0.2315" data-browser-id="1e082982-4fd7-4d6a-8a19-74933b87fb29" data-call-tree-id="AAYxFgFLwseJG2523T1drA==" data-dfp-member-lix-treatment="control" data-disable-jsbeacon-pagekey-suffix="false" data-dna-member-lix-treatment="enabled" data-enable-page-view-heartbeat-tracking="" data-human-member-lix-treatment="control" data-member-id="0" data-multiproduct-name="jobs-guest-frontend" data-page-instance="urn:li:page:d_jobs_guest_search;Ne/XXT0xS26MiFXPCxXLNw==" data-service-name="jobs-guest-frontend" data-should-use-full-url-in-pve-path="true" data-sync-apfc-cb-lix-treatment="enabled" data-sync-apfc-headers-lix-treatment="control" id="config"/>
  <link href="https://ph.linkedin.com/jobs/python-developer-jobs" rel="canonical"/>
  <!-- -->
  <!--

In [251]:
soup.

<html lang="tl"><head>
<meta content="d_jobs_guest_search" name="pageKey"/>
<!-- --> <meta content="urlType=jserp_canonical_other;emptyResult=false" name="linkedin:pageTag"/>
<meta content="tl_PH" name="locale"/>
<meta data-app-version="2.0.2315" data-browser-id="6d021328-e517-4ebd-8d65-7a80de2f6e76" data-call-tree-id="AAYxFnBgr/IVhk725xphZw==" data-dfp-member-lix-treatment="control" data-disable-jsbeacon-pagekey-suffix="false" data-dna-member-lix-treatment="enabled" data-enable-page-view-heartbeat-tracking="" data-human-member-lix-treatment="control" data-member-id="0" data-multiproduct-name="jobs-guest-frontend" data-page-instance="urn:li:page:d_jobs_guest_search;8/pDPGcwTXmGHCcs+GHFRQ==" data-service-name="jobs-guest-frontend" data-should-use-full-url-in-pve-path="true" data-sync-apfc-cb-lix-treatment="enabled" data-sync-apfc-headers-lix-treatment="control" id="config"/>
<link href="https://ph.linkedin.com/jobs/python-developer-jobs" rel="canonical"/>
<!-- --><!-- -->
<!-- -->
<!-- 

In [126]:
job_list[1].find_element(By.XPATH, "//a[@data-tracking-control-name='public_jobs_jserp-result_search-card']").get_attribute('href')

'https://ph.linkedin.com/jobs/view/junior-python-developer-at-shiperp-4149413701?position=1&pageNum=0&refId=sfkVnb0sd7UqEnZ1CJnldw%3D%3D&trackingId=ONHjcrLq5BnUteHIqYbj2g%3D%3D'

In [211]:
print(BeautifulSoup(job_list[0].get_attribute('outerHTML'), 'html.parser').prettify())

<li>
 <div class="base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card" data-column="1" data-entity-urn="urn:li:jobPosting:4149413701" data-reference-id="L4Y7K5211cBaLHxvZWxQXw==" data-row="1" data-tracking-id="vL9Bj37PPk3oClIfzfOtew==">
  <a class="base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]" data-tracking-client-ingraph="" data-tracking-control-name="public_jobs_jserp-result_search-card" data-tracking-will-navigate="" href="https://ph.linkedin.com/jobs/view/junior-python-developer-at-shiperp-4149413701?position=1&amp;pageNum=0&amp;refId=L4Y7K5211cBaLHxvZWxQXw%3D%3D&amp;trackingId=vL9Bj37PPk3oClIfzfOtew%3D%3D">
   <span class="sr-only">
    Junior Python Developer
   </span>
  </a>
  <div class="search-entity-media">
   <img alt="" aria-busy="false" class="artdeco-entity-image artdeco-entity-image--square-4 lazy-loaded" data-ghost-classes="artdeco-entity-image--ghost" data-ghos